In [1]:
import re
from dotenv import load_dotenv
from openai import OpenAI

## Language Model Setup

In [2]:
load_dotenv()

client = OpenAI()
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role': 'user', 'content': 'Hi there'}]
)

response

ChatCompletion(id='chatcmpl-9Wy5gexISSu4dW1B3CuoVdFe1emdN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1717644512, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))

In [3]:
response.choices[0].message.content

'Hello! How can I assist you today?'

## ReAct Agent Class
![ReAct Workflow](https://storage.googleapis.com/gweb-research2023-media/original_images/cca912e7fbe652676302383247087e22-Screen20Shot202022-11-0820at208.53.4920AM.png)

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = [{'role': 'system', 'content': system}]
    
    def __call__(self, message):
        self.messages.append({'role': 'user', 'content': message})
        result = self._call_model()
        self.messages.append({'role': 'assistant', 'content': result})
        return result

    def _call_model(self):
        result = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=self.messages
        )
        return result.choices[0].message.content

In [5]:
sys_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [6]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [8]:
bot = Agent(sys_prompt)
result = bot("How much does a toy poodle weigh?")
print(result)

Thought: I should use the average_dog_weight function to find out the weight of a Toy Poodle.
Action: average_dog_weight: Toy Poodle
PAUSE


In [9]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [10]:
next_prompt = "Observation: {}".format(result)
bot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs.'

In [11]:
bot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [12]:
bot2 = Agent(sys_prompt)

question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
bot2(question)

'Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight action and then calculate their combined weight.\n\nAction: average_dog_weight: Border Collie\nPAUSE'

In [13]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [14]:
bot2(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [15]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [16]:
bot2(next_prompt)

'Action: calculate: 37 + 20\nPAUSE'

In [17]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [18]:
bot2(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Automate the loop

In [21]:
action_re = re.compile("^Action: (\w+): (.*)$")   # python regular expression to selection action

def query(question, max_turns=5):
    i = 0
    bot = Agent(sys_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/pk/4r2klxbd3lgg7t_6g_35zxsm0000gn/T/ipykernel_4201/1336232285.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile("^Action: (\w+): (.*)$")   # python regular expression to selection action


In [22]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
query(question)

Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight function.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
